In [3]:
import os
import shutil
from tqdm import tqdm
import pandas as pd
import file_function as ff

In [ ]:
############################################
#일단 폴더 별로 해야 수작업 들어가야 하는 항목을 최소화하기 좋다.
#중복검사는 지점 통합 후에 또 해야 한다. / 혹은 통합 후에 한 번에 하자
############################################
# 사이즈가 젤 중요. 사이즈가 같으면 이름과 수정일은 거의 같음. 
# pdf류라 수정일이 달라질 이유가 없음. 더러 다른 날 같은 파일을 스캔해서 수정일이 다른 경우가 있는데 이때는 어차피 사이즈도 다름.
# 이걸 찾기는 어려움
# 이름(채무자키)이 아예 다른 경우가 있는데 이건 애초에 이름이나 폴더가 잘못된 것.
# 보이면 올바르게 수정할 수 있지만, 일부러 찾기란 어려움
# 사이즈가 같은 거 중에 혹시 다른 거 있는지 보조용으로 체크섬사용하는 것
# 체크섬만 사용해봐야 시간만 더 오래 걸릴 뿐 결과는 사이즈랑 같이 본 것과 동일
# 등초본의 경우 채무자키가 전혀 달라도(채권사가 아예다름) 동일한 파일이 있을 수 있고 이 경우는 유효함.

In [12]:
import os
import hashlib
import collections


def compute_partial_hash_of(file_path, seek_pos, chunk_size):
    with open(file_path, "rb") as f:
        f.seek(seek_pos)
        data = f.read(chunk_size)
        if not data:
            return  # no more yield
        hashes = [hashlib.sha256(), hashlib.md5()]
        hashes[0].update(data)
        hashes[1].update(data)
        yield "_".join(map(lambda y: y.hexdigest(), hashes))


def make_challenge(file_path):
    chunk_size = 1048576  #단위 byte.. 즉 1MB 살펴보겠다
    file_size = os.path.getsize(file_path)
    yield file_size
    yield from compute_partial_hash_of(file_path, 0, chunk_size)
    yield from compute_partial_hash_of(file_path, file_size - chunk_size, chunk_size)


def fuzzy_group_files(target_directory):
    target_files = []
    for root, _, files in os.walk(target_directory):
        target_files.extend(map(lambda x: os.path.join(root, x), files))

    groups, challenge, challenge_result = init_challenge(target_files)

    found_challenge = True
    while found_challenge:
        found_challenge = False
        for _, v in groups.items():
            if len(v) > 1:
                for file_path in v:
                    if challenge[file_path] is not None:
                        try:
                            challenge_result[file_path].append(
                                next(challenge[file_path]))
                            found_challenge = True
                        except StopIteration:
                            challenge[file_path] = None
                            pass
        groups = regroup(challenge_result)
        print(collections.Counter(map(lambda x: len(x), groups.values())))
    return groups


def regroup(challenge_result):
    groups = {}  # Regroup items
    for file_path, v in challenge_result.items():
        key = "/".join(map(lambda x: str(x), v))
        if key not in groups:
            groups[key] = []
        groups[key].append(file_path)
    return groups


def init_challenge(target_files):
    groups = {"": []}
    challenge = {}
    challenge_result = {}
    for file_path in target_files:
        challenge[file_path] = make_challenge(file_path)
        challenge_result[file_path] = []
        groups[""].append(file_path)
    return groups, challenge, challenge_result


In [21]:
target_directory = r"C:\Users\SL\Desktop\test"
groups = fuzzy_group_files(target_directory)
for _, v in groups.items():
    if len(v) > 1:
        print("### Group ###")
        print("\n".join(v))

Counter({1: 1, 2: 1})
Counter({1: 1, 2: 1})


OSError: [Errno 22] Invalid argument

In [1]:
file_path = r'C:\Users\SL\Desktop\test\20407822_김대식_초본.pdf'
with open(file_path, "rb") as f:
    data = f.read(1048576)

In [2]:
data

b'%PDF-1.4\n%\xe2\xe3\xcf\xd3\r\n\r\n1 0 obj\n<</Type /Page\n/Parent 2 0 R\n/MediaBox [ 0 0 595.000 841.000 ]\n/Resources <</XObject 3 0 R /ProcSet [ /PDF /Text /ImageB /ImageC /ImageI ]>>/Contents [ 4 0 R ]\n/Rotate 0\n>>\r\nendobj\n5 0 obj\n<</Type /XObject\n/Subtype /Image\n/Name /JI1a\n/Width 1653\n/Height 2338\n/BitsPerComponent 8\n/ColorSpace /DeviceRGB\n/Filter /DCTDecode\n/Length 6 0 R\n>>\r\nstream\r\n\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x01\x00\xc8\x00\xc8\x00\x00\xff\xfe\x00\r28BW-5e Q76\xff\xdb\x00\x84\x00\x05\x05\x06\x07\x06\x06\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\x01\x05\x06\x06\t\x08\t\r\x0b\x0b\r\x13\x10\r\x10\x13\x1b\x17\x14\x14\x17\x1b"\x1e\x1b\x17\x1b\x1e"*&"\x1e\x1e"&*-)&"&)-2-))-222-222222222222\xff\xc0\x00\x11\x08\t"\x06u\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x01\xa2\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\

A
B
C


In [1]:
##########작업 경로 설정하기 : 엑셀에서 미리 \를 /로 바꿔서 입력할 것
os.chdir("C:/Users/SL/Downloads/부산 공유폴더")
path = os.getcwd()
path

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: 'C:/Users/SL/Downloads/부산 공유폴더'

In [19]:
z="/"

In [ ]:
####### 모든 파일에 중복파일 횟수, 생성,수정 날짜를 표시하고 엑셀로

In [4]:
ff.all_files(path)

542it [00:18, 29.77it/s]


In [9]:
###### 파일 확장자
extension_set = set()

for root, dirs, files in os.walk(path) :

    for f in files :
        extension_set.add(os.path.splitext(f)[1])

In [10]:
extension_set

{'.PDF',
 '.csv',
 '.db',
 '.ini',
 '.pdf',
 '.png',
 '.teamup',
 '.tif',
 '.txt',
 '.xls',
 '.xlsx',
 '.zip'}

In [20]:
os.chdir("//192.168.10.72/공유폴더")
path = os.getcwd()
path

'\\\\192.168.10.72\\공유폴더'

In [21]:
for root, dirs, files in os.walk(path) :

    for f in files :
        extension_set.add(os.path.splitext(f)[1])
extension_set

{'',
 '.210607',
 '.7z',
 '.JPG',
 '.PDF',
 '.PNG',
 '.TIF',
 '.TIF_20090518153511',
 '.TIf',
 '.bkg',
 '.bmp',
 '.csv',
 '.db',
 '.dll',
 '.doc',
 '.docx',
 '.download',
 '.egg',
 '.exe',
 '.gul',
 '.hwp',
 '.ico',
 '.ini',
 '.jpg',
 '.lnk',
 '.log',
 '.mht',
 '.msi',
 '.partial',
 '.pdf',
 '.pdf 김순한 양통',
 '.png',
 '.rtf',
 '.teamup',
 '.tiF',
 '.tif',
 '.tif_20120906100617',
 '.tiff',
 '.tmp',
 '.txt',
 '.url',
 '.wav',
 '.xls',
 '.xlsm',
 '.xlsx',
 '.zip',
 '.이성훈의 보증인 백성기pdf'}

In [4]:
a= os.path.abspath("새 텍스트 문서.txt")
print(a)


D:\부산 정리\D_test\새 텍스트 문서.txt


In [6]:
path = os.path.dirname(a) 
print(path)

D:\부산 정리\D_test


In [9]:
a = files_list[1]

In [14]:
print(len(files_list[1]))

3523


In [30]:
import pandas as pd

In [37]:
col=['root', 'dirs', 'files']

In [38]:
df = pd.DataFrame(zip(root_list, dirs_list, files_list), columns=col)

In [39]:
df.head(5)

,root,dirs,files
0,\\192.168.10.72\공유폴더,"[2_파일명변경완료, @◆통합파일◆@, Log, ■매각 계약서, ■양도통지서 스캔,...",[desktop.ini]
1,\\192.168.10.72\공유폴더\2_파일명변경완료,[],"[20407790 하회식 원인서류1.pdf, 20407790 하회식 원인서류2.pd..."
2,\\192.168.10.72\공유폴더\@◆통합파일◆@,"[(1.DNP_헬프), (2.한울가람_헬프), (3.산은캐피탈_헬프), (4.흥국화...",[]
3,\\192.168.10.72\공유폴더\@◆통합파일◆@\(1.DNP_헬프),"[-이름, 사건번호 안뜸, 1.매입전 집행권원, 10.부동산가압류 결정문, 11.기...",[]
4,"\\192.168.10.72\공유폴더\@◆통합파일◆@\(1.DNP_헬프)\-이름, ...",[],"[김향숙 2018카기866.pdf, 김현지 2019타기190.pdf, 나신기_양통...."


In [15]:
import csv

In [40]:
os.chdir("C:/Users/SL/Desktop/python_work")
os.getcwd()

'C:\\Users\\SL\\Desktop\\python_work'

In [43]:
df.to_csv('files_name.csv', encoding='utf-8-sig')